In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import numpy as np

In [ ]:
Stock_A = "SPLG"
Stock_B = "SPY"
stocks = [Stock_A,Stock_B]

In [ ]:
StockPrices = yf.download(stocks,"2024-01-01","2025-01-01",interval="1h")['Close']
StockPrices.dropna(inplace=True)


In [ ]:
StockPrices.corr()

In [ ]:
StockPrices['Rt']=StockPrices[Stock_A]/StockPrices[Stock_B]
StockPrices['Rt1']=StockPrices['Rt'].shift(1)
StockPrices['DeltaRt']=StockPrices['Rt']-StockPrices['Rt1']
StockPrices

In [ ]:
StockPrices = StockPrices.dropna()
StockPrices

In [ ]:
model=sm.ols(data=StockPrices, formula="DeltaRt ~ Rt1").fit(cov_type='HC3') 
model.summary() 

In [ ]:
c=model.params[0]
c

In [ ]:
d = -model.params[1]
d

In [ ]:
NewStockPrices=yf.download(stocks,"2025-01-01","2025-06-14",interval="1h")['Close']
NewStockPrices.dropna(inplace=True)


In [ ]:
NewStockPrices['Rt']=NewStockPrices[Stock_A]/NewStockPrices[Stock_B]
NewStockPrices['Rt1']=NewStockPrices['Rt'].shift(1)
NewStockPrices['DeltaRt']=NewStockPrices['Rt']-NewStockPrices['Rt1']
NewStockPrices = NewStockPrices.dropna()
NewStockPrices

In [ ]:
threshold = c / d
NewStockPrices['A_Position'] = 0
NewStockPrices['B_Position'] = 0
NewStockPrices['Portfolio_Value']=0
NewStockPrices['Delta_A_Position'] = 0
NewStockPrices['Delta_B_Position'] = 0
NewStockPrices['Cash_Position'] = 0
NewStockPrices['Cash_Flow'] = 0
NewStockPrices['Cumulative_Number_of_transaction'] = 0
threshold

In [ ]:
for i in range(1, len(NewStockPrices)):
    Rt = NewStockPrices['Rt'].iloc[i]

    if Rt > threshold and NewStockPrices['A_Position'].iloc[i-1]>=0:
        NewStockPrices['A_Position'].iloc[i] = -100/NewStockPrices[Stock_A].iloc[i]
        NewStockPrices['B_Position'].iloc[i] = 100/NewStockPrices[Stock_B].iloc[i]
    elif Rt < threshold and NewStockPrices['A_Position'].iloc[i-1]<=0:  
        NewStockPrices['A_Position'].iloc[i] = 100/NewStockPrices[Stock_A].iloc[i]
        NewStockPrices['B_Position'].iloc[i] = -100/NewStockPrices[Stock_B].iloc[i]
    else:
        NewStockPrices['A_Position'].iloc[i] = NewStockPrices['A_Position'].iloc[i-1]  # Hold position
        NewStockPrices['B_Position'].iloc[i] = NewStockPrices['B_Position'].iloc[i-1]  
    

    NewStockPrices['Delta_A_Position'].iloc[i] = NewStockPrices['A_Position'].iloc[i] - NewStockPrices['A_Position'].iloc[i-1]
    NewStockPrices['Delta_B_Position'].iloc[i] = NewStockPrices['B_Position'].iloc[i] - NewStockPrices['B_Position'].iloc[i-1]
    

    if NewStockPrices['A_Position'].iloc[i] != NewStockPrices['A_Position'].iloc[i-1]:
        NewStockPrices['Cash_Position'].iloc[i] = (NewStockPrices['Delta_A_Position'].iloc[i] * -NewStockPrices[Stock_A].iloc[i]) + (NewStockPrices['Delta_B_Position'].iloc[i] * -NewStockPrices[Stock_B].iloc[i]) + NewStockPrices['Cash_Position'].iloc[i-1]
    else:
        NewStockPrices['Cash_Position'].iloc[i] = NewStockPrices['Cash_Position'].iloc[i-1]
                             

    NewStockPrices['Portfolio_Value'].iloc[i] = (NewStockPrices['A_Position'].iloc[i] * NewStockPrices[Stock_A].iloc[i]) + (NewStockPrices['B_Position'].iloc[i] * NewStockPrices[Stock_B].iloc[i]) + NewStockPrices['Cash_Position'].iloc[i] 
    
    NewStockPrices['Cash_Flow'].iloc[i] = NewStockPrices['Cash_Position'].iloc[i] - NewStockPrices['Cash_Position'].iloc[i-1]

    if NewStockPrices['Cash_Flow'].iloc[i] !=0:
        NewStockPrices['Cumulative_Number_of_transaction'].iloc[i] = NewStockPrices['Cumulative_Number_of_transaction'].iloc[i-1] + 1
    else:
        NewStockPrices['Cumulative_Number_of_transaction'].iloc[i] = NewStockPrices['Cumulative_Number_of_transaction'].iloc[i-1]
                                              


In [ ]:
NewStockPrices

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(NewStockPrices.index, NewStockPrices['Portfolio_Value'], label='Portfolio Value', color='blue')
plt.title('Dollar Value of Long-Short Portfolio Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.legend()
plt.grid(False)
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(NewStockPrices.index, NewStockPrices['Cash_Flow'], label='Cash Flow', color='blue')
plt.title('Cash Flow Over Time')
plt.xlabel('Date')
plt.ylabel('Cash ')
plt.legend()
plt.grid(False)
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(NewStockPrices.index, NewStockPrices['Cumulative_Number_of_transaction'], label='Number_of_transaction', color='blue')
plt.title('Cumulative_Number_of_transaction Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Transactions')
plt.legend()
plt.grid(False)

In [ ]:
returns = NewStockPrices['Portfolio_Value']/NewStockPrices['Portfolio_Value'].shift(1)-1
returns.replace([np.inf, -np.inf],np.nan,inplace=True)
mu = returns.mean()*252
s = returns.std()*252**0.5
mu/s

In [ ]:
returns